# Mixture Models and Expectation-Maximization

<hr>

**A gentle introduction to Expectation-Maximization (EM) Algorithm**<br>

While a "hard" clustering algorithm like k-means or k-medoids can only provide a cluster ID for each data point, the *EM* algorithm, along with the model driving its equations, can provide the posterior probability ("soft" assignments) that every data point belongs to any cluster.

For a given data point, the probabilistic estimate of it belonging to any cluster can be represented as a multinomial:

$x \sim P(x | \mu_j, \sigma_{j}^2)$ 

where 

- $j = 1, \dots, K$
- $K$ is the number of clusters

<hr>

**Gaussian Mixture Model (GMM)**

This is a generative model for data $x \in \mathbb{R}^d$ and is defined by the following set of parameters:

1. $K$, the number of mixture components
2. $d$-dimensional Gaussian $\mathcal{N} (\mu_j, \sigma_j^2)$ for every $j = 1, \dots, K$
3. $p_1, \dots, p_K$, a set of mixture weights which sums to 1, where $\hat{p_j} = \frac{\hat{n_j}}{n}$ and $\hat{n_j}$ is the number of data points in the cluster (multinomial MLE)

The parameters can be collectively represented as:

$\theta = \{ p_1, \dots, p_K, \mu_1, \dots, \mu_K, \sigma_1^2, \dots, \sigma_K^2 \}$

The likelihood of observing a point $x$ across all clusters in a GMM is given as:

$p(x | \theta) = \sum_{j=1}^{K} p_j \mathcal{N} (x, \mu_j, \sigma_j^2)$

Given a set of points, $S_n$, we maximize the likelihood of $\theta$ by finding the likelihood of a point across all clusters and taking the joint probability of all points:

$p(S_n | \theta) = \prod_{i=1}^{n} \sum_{j=1}^{K} p_j \mathcal{N} (x; \mu_j, \sigma_j^2)$

There is no closed-form solution to finding the parameter set $\theta$ that maximizes the likelihood and therefore the EM algorithm is necessary.

****

**Deriving the EM Algorithm**

The high-level idea follows two steps:

1. Compute the probabilistic soft assignment of points to different clusters given a random initialization (*or k-means*) of the set of parameters characterizing the GMM
2. Then compute the likelihood of observing this set of points given the probabilistic weights under the random set of parameters

... and we continue these two steps until the parameters converges to the maximum likelihood.

<img alt="EM Iterations" src="assets/em_iterations.png" width="500">

Formally, these are the steps to EM:

1. Randomly initialize the parameters, $\theta = \{p_1, \dots, p_K, \mu_1, \dots, \mu_K, \sigma_1^2, \dots, \sigma_K^2 \}$

2. **E-step**: Finds the posterior probability that a certain point $x_i$ belonging to a cluster $j$

    $p(j | i) = \frac{p_j \mathcal{N}(x^{(i)}; \mu_j, \sigma_j^2 \cdot I)}{p(x|\theta)}$


3. **M-step**: Re-estimate the parameters by maximizing the likelihood

    $\hat{n_j} = \sum_{i=1}^{n} p(j|i)$
    
    $\hat{p_j} = \frac{\hat{n_j}}{n}$
    
    $\hat{\mu_j} = \frac{1}{\hat{n_j}} \sum_{i = 1}^{n} \hat{p_j} x_i$
    
    $\hat{\sigma_j^2} = \frac{1}{\hat{n_j}d} \sum_{i=1}^{n} \hat{p_j} \cdot \Vert x_i - \hat{\mu_j} \Vert^2$
    
    
4. Repeat iteratively until convergence

The EM algorithm is only guaranteed to converge to a local minima. In reality, initialization in step (1) is typically applied with K-means to find initial cluster centers of $K$ clusters and use the global variance of the dataset as the initial variance of all the $K$ clusters.

Generally, GMM can be extended to a mixture component with a general covariance matrix $\Sigma_j$

<hr>

# Basic code
A `minimal, reproducible example`

*Question 1*

Assume that the initial means and variances of two clusters in a GMM are as follows:

$\mu = \begin{bmatrix} -3 \\ 2 \end{bmatrix}$

$\sigma_1^2 = \sigma_2^2 = 4$

$p_1 = p_2 = 0.5$

$X = \begin{bmatrix} 0.2 & -0.9 & -1 & 1.2 & 1.8 \end{bmatrix}^T$

In [1]:
# E-step: 
# Compute the posterior probabilities of each observation in cluster 1

import numpy as np
from scipy.stats import norm

mu = np.array([-3, 2])
sigma = np.array([2, 2])
p = np.array([0.5, 0.5])
X = np.array([0.2, -0.9, -1, 1.2, 1.8])

posterior1 = []

for x in X:
    m1, m2 = mu[0], mu[1]
    sigma1, sigma2 = sigma[0], sigma[1]
    p1, p2 = p[0], p[1]

    l1 = norm.pdf(x, loc = m1, scale = sigma1)
    l2 = norm.pdf(x, loc = m2, scale = sigma2)
    denom = l1*p1+l2*p2
    posterior = l1*p1/denom
    posterior1.append(posterior)
    
posterior1

[0.29421497216298875,
 0.6224593312018545,
 0.6513548646660543,
 0.1066905939456512,
 0.053403329799824234]

In [2]:
# M-step:
# Compute the updated parameters corresponding to cluster 1

n1_hat = sum(posterior1)
p1_hat = n1_hat/X.shape[0]
mu1_hat = 1/n1_hat * sum([posterior_i * x_i for posterior_i, x_i in zip(posterior1, X)])
var_hat = 1/n1_hat * sum([posterior_i * (x_i - mu1_hat)**2 for posterior_i, x_i in zip(posterior1, X)])

print(n1_hat, p1_hat, mu1_hat, var_hat)

1.728123091776373 0.3456246183552746 -0.5373289474340417 0.5757859076870628
